In [1]:
import numpy as np
import pandas as pd
import itertools as it

In [2]:
class Die():
    '''
    Docstring
    '''
    def __init__(self, face_symbols):
        
        # Test for numpy array
        if not isinstance(face_symbols, np.ndarray):
            raise TypeError('Face Argument must be NumPy array')
          
        # Tests that array is strings or numbers
        if face_symbols.dtype.char not in ['U', 'l', 'd']:
            raise TypeError('Faces must be strings or numbers')
        
        # Tests for unique faces
        if len(set(face_symbols)) != len(face_symbols):
            raise ValueError('Face values must be distinct')
        
        # Initialize weights as 1.0 for each face
        self.faces = face_symbols
        self.weights = [1.0 for i in face_symbols]
        
        # Save faces and weights in private data frame
        self._faces_weights = pd.DataFrame(self.weights, self.faces, columns=['weight'])
        
    def change_side_weight(self, face, new_weight):
        '''
        '''
        if face not in self._faces_weights.index:
            raise IndexError('That face does not exist on this die')
        if type(new_weight) not in [int, float]:
            raise ValueError('Weight is not a valid type')
        
        self._faces_weights.loc[face] = new_weight
        
    def roll(self, n = 1):
        '''
        '''
        results = []
        self.probs = [i/sum(self._faces_weights.weight) for i in self._faces_weights.weight]
        
        for i in range(n):
            result = self._faces_weights.sample(weights = self.probs).index.values[0]
            results.append(result)
            
        return results
        
    def current_state(self):
        '''
        '''
        return self._faces_weights
        
class Game():
    
    def __init__(self, die_list):
        self.die_list = die_list
    
    def play(self, n):
        '''
        '''
        self.df_index = []
        for i in range(n):
            self.df_index.append('Roll ' + str(i + 1))
        
        self._play_results = pd.DataFrame([], self.df_index)
        
        for die in self.die_list:
            self._play_results.insert(self.die_list.index(die), self.die_list.index(die) + 1, die.roll(n))
    
    def last_round(self, form = 'wide'):
        '''
        '''
        if form == 'narrow':
            return self._play_results.stack().to_frame('Value')
        elif form == 'wide':
            return self._play_results
        else:
            raise ValueError('Must request for a "narrow" or "wide" table')
        
class Analyzer():
    def __init__(self, game_object):
        if not isinstance(game_object, Game):
            raise ValueError('Passed value is not a Game object')
        
        self.game = game_object
        
        self.outcome = game_object._play_results
    
    def jackpot(self):
        '''
        '''
        count = 0
        for i in self.outcome.nunique(axis=1) == 1:
            if i == True:
                count += 1
        
        return count
    
    def face_count(self):
        '''
        '''
        return pd.DataFrame(self.outcome).apply(pd.Series.value_counts, axis = 1).fillna(0)
    
    def combo_count(self, sort = True):
        '''
        '''
        sorted_outcome = self.outcome.apply(sorted, axis = 1, result_type = 'broadcast')
        return sorted_outcome.value_counts(sort = sort).to_frame('Combination Count')
    
    def permutation_count(self, sort = True):
        '''
        '''
        return self.outcome.value_counts(sort = sort).to_frame('Permutation Count')

In [3]:
faces = np.array([1, 2, 3, 4, 5, 6])

In [4]:
die1 = Die(faces)
die2 = Die(faces)
die3 = Die(faces)
die4 = Die(faces)
die5 = Die(faces)
die6 = Die(faces)

In [5]:
die_list = [die1, die2]

In [6]:
jeff = Game(die_list)

In [7]:
die1.current_state()

,weight
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0


In [8]:
jeff.play(20)

In [9]:
jeff._play_results

,1,2
Roll 1,6,3
Roll 2,4,5
Roll 3,4,1
Roll 4,1,4
Roll 5,6,4
Roll 6,5,2
Roll 7,5,5
Roll 8,6,1
Roll 9,5,4
Roll 10,3,4


In [10]:
jeff.last_round()

,1,2
Roll 1,6,3
Roll 2,4,5
Roll 3,4,1
Roll 4,1,4
Roll 5,6,4
Roll 6,5,2
Roll 7,5,5
Roll 8,6,1
Roll 9,5,4
Roll 10,3,4


In [11]:
jeff_analyzer = Analyzer(jeff)

In [12]:
jeff_analyzer.jackpot()

4

In [13]:
jeff_analyzer.face_count()

,1,2,3,4,5,6
Roll 1,0.0,0.0,1.0,0.0,0.0,1.0
Roll 2,0.0,0.0,0.0,1.0,1.0,0.0
Roll 3,1.0,0.0,0.0,1.0,0.0,0.0
Roll 4,1.0,0.0,0.0,1.0,0.0,0.0
Roll 5,0.0,0.0,0.0,1.0,0.0,1.0
Roll 6,0.0,1.0,0.0,0.0,1.0,0.0
Roll 7,0.0,0.0,0.0,0.0,2.0,0.0
Roll 8,1.0,0.0,0.0,0.0,0.0,1.0
Roll 9,0.0,0.0,0.0,1.0,1.0,0.0
Roll 10,0.0,0.0,1.0,1.0,0.0,0.0


In [16]:
jeff_analyzer.permutation_count(False)

Permutation Count
1 2                   
1 4                  1
2 2                  1
  4                  1
  5                  1
  6                  1
3 3                  1
  4                  1
  5                  1
4 1                  1
  2                  1
  5                  1
5 1                  1
  2                  1
  3                  1
  4                  1
  5                  1
6 1                  1
  3                  1
  4                  1
  6                  1

In [17]:
jeff_analyzer.combo_count(False)

Combination Count
1 2                   
1 4                  2
  5                  1
  6                  1
2 2                  1
  4                  2
  5                  2
  6                  1
3 3                  1
  4                  1
  5                  2
  6                  1
4 5                  2
  6                  1
5 5                  1
6 6                  1

In [116]:
die1 = Die(np.array([1, 2, 3]))
        
expected = pd.DataFrame([1.0, 1.0, 1.0], [1, 2, 3], columns = ['weight'])

die1.current_state().equals(expected)


True

In [148]:
die1 = Die(np.array([1, 2, 3]))
die2 = Die(np.array([1, 2, 3]))
die_list = [die1, die2]
        
game1 = Game(die_list)
game1.play(3)
Analyzer(game1).outcome

,1,2
Roll 1,2,2
Roll 2,2,2
Roll 3,1,1


In [150]:
game = 5
Analyzer(game)

ValueError: Passed value is not a Game object

In [156]:
expected = pd.DataFrame([[1, 1], [1, 0]], [1, 2], columns = [1, 2])

In [157]:
expected

,1,2
1,1,1
2,1,0


In [220]:
die1 = Die(np.array([1, 2, 3]))
die2 = Die(np.array([1, 2, 3]))
die_list = [die1, die2]
        
game1 = Game(die_list)
game1.play(3)
        
analyzer1 = Analyzer(game1)
analyzer1.outcome = pd.DataFrame(np.array([[1, 1, 3, 3, 5], [1, 1, 2, 2, 4]]), columns = [1, 2, 3, 4, 5])

In [214]:
analyzer1.outcome

,1,2,3,4,5
0,1,1,3,3,5
1,1,1,2,2,4


In [216]:
analyzer1.face_count()

,1,2,3,4,5
0,2.0,0.0,2.0,0.0,1.0
1,2.0,2.0,0.0,1.0,0.0


In [218]:
test = pd.DataFrame(np.array([[2.0, 0.0, 2.0, 0.0, 1.0], [2.0, 2.0, 0.0, 1.0, 0.0]]), columns = [1, 2, 3, 4, 5])

In [221]:
analyzer1.face_count().equals(test)

True

In [166]:
die1 = Die(np.array([1, 2, 3]))
die2 = Die(np.array([1, 2, 3]))
die_list = [die1, die2]
        
game1 = Game(die_list)
game1.play(3)
        
expected = (3, 2)

game1.last_round().shape == expected

True

In [222]:
die1 = Die(np.array([1, 2, 3]))
die2 = Die(np.array([1, 2, 3]))
die_list = [die1, die2]
        
game1 = Game(die_list)
game1.play(3)
        
analyzer1 = Analyzer(game1)
analyzer1.outcome = pd.DataFrame(np.array([[1, 2, 3], [3, 2, 1]]), columns = [1, 2, 3])

In [236]:
analyzer1.combo_count().shape == (1, 1)

True

In [237]:
analyzer1.permutation_count().shape == (2, 1)

True

In [253]:
die1 = Die(np.array([1, 2, 3]))
die1.change_side_weight(1, 4)
        
expected = 4
        
#self.assertEqual(die1._faces_weights.loc[1], expected)

In [254]:
die1.current_state()

,weight
1,4.0
2,1.0
3,1.0


In [282]:
die1._faces_weights.weight[1] == 4

True